In [2]:
#import packages
import os
import boto3
from botocore.exceptions import ClientError

### File Upload into the bucket

In [14]:
s3 = boto3.client('s3')

bucket_name = 's3demo00001'
actual_file_name = 'rowan-heuvel-YJZzNdMLzF8-unsplash.jpg'

path_to_file = os.path.join((r'C:\Users\atul1\Pictures\AWS'),actual_file_name)

#upload of image file
s3.upload_file(
    Filename = path_to_file,
        Bucket = bucket_name,
        Key = actual_file_name)


In [22]:
def local_file_upload(file):

    """ This function help to upload local files into the s3 bucket

    Parameters:
    -----------
    bucket_name : Name of the bucket

    Key         : name of the object to be uploaded
    
    filename    : path of the object to be uploaded

    Returns:
    --------
    None

    """

    bucket_name = 's3demo00001'
    Key = file
    filename = os.path.join((r'C:\Users\atul1\Pictures\AWS'),file)

    s3.upload_file(
        Filename = filename,
        Bucket = bucket_name,
        Key = Key,
    )

In [19]:
#Upload multiple files in Bucket
for fls in os.listdir(r'C:\Users\atul1\Pictures\AWS'):
    local_file_upload(fls)

### List files in Bucket

In [34]:
list_response = s3.list_objects(
    Bucket = 's3demo00001',
)

In [35]:
for fls_no in range(len(list_response['Contents'])):
    fls = list_response['Contents'][fls_no]['Key']
    print(fls)

arfan-abdulazeez-GVYRkT5f1tA-unsplash.jpg
rowan-heuvel-YJZzNdMLzF8-unsplash.jpg
sepp-rutz-tskqMngoHSA-unsplash.jpg


### Delete files in Bucket

In [36]:
s3.delete_object(
    Bucket = 's3demo00001',
    Key = 'scott-graham-5fNmWej4tAA-unsplash.jpg'
)

{'ResponseMetadata': {'RequestId': '4S9C20QD2YTY3TT4',
  'HostId': 'XqfhfPUvmRC6+HDdedoaRRwoV+Q4EHR+I1u2Lw+/5YMeWnzchqiOTiLYi0EiQt5MlhVF9ahxp9M=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'XqfhfPUvmRC6+HDdedoaRRwoV+Q4EHR+I1u2Lw+/5YMeWnzchqiOTiLYi0EiQt5MlhVF9ahxp9M=',
   'x-amz-request-id': '4S9C20QD2YTY3TT4',
   'date': 'Sun, 13 Jun 2021 10:47:23 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [42]:
#Download a file
s3.download_file(
    Filename = '.\sepp-rutz-tskqMngoHSA-unsplash.jpg',
    Bucket = 's3demo00001',
    Key = 'sepp-rutz-tskqMngoHSA-unsplash.jpg'
)

## ACLS
### In AWS there are 4 ways to provide Access to objects
* IAM 
* Bucket Policy
* ACL
* Presigned URLs

Now ACL can be of two types:

1: Public Read 

2: Private

In [ ]:
#Set an object as Public - Read 

s3.put_object_acl(
    Bucket = 's3demo00001',
    Key = 'rowan-heuvel-YJZzNdMLzF8-unsplash.jpg',
    ACL = 'public-read'
)

In [47]:
#Setting ACLs on upload (Upload a file using 'public-read')

s3.upload_file(
    Bucket = 's3demo00001',
    Filename = '.\sales_data_sample.csv',
    Key = 'sales_data_sample.csv',
    ExtraArgs={'ACL':'public-read'}
)

#s3 object URL Template

https://{bucket}.s3.amazonaws.com/{key}

URL for Key 'sales_data_sample.csv'

In [53]:
url = "https://s3demo00001.s3.amazonaws.com/sales_data_sample.csv"

In [59]:
import pandas as pd

#here encoding other than utf-8 is used
df = pd.read_csv(url, encoding='Latin-1')
df.head(5)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [60]:
#upload a data file in s3 bucket and then trying to access without ACL

s3.upload_file(
    Filename= 'sales_data_02.csv',
    Bucket='s3demo00001',
    Key = 'sales_data_02.csv'
)

In [ ]:
#generate a url of the above file and try loading into a dataframe

url_02 = 'https://s3demo00001.s3.amazonaws.com/sales_data_02.csv'

df_02 = pd.read_csv(url_02, encoding='Latin-1')
df_02.head(5)

#here we get 403 Error

In [ ]:
#Now will make the sales_data_02 access to public

s3.put_object_acl(
    Key = 'sales_data_02.csv',
    Bucket = 's3demo00001',
    ACL ='public-read'
)

In [63]:
url_03 = 'https://s3demo00001.s3.amazonaws.com/sales_data_02.csv'

df_03 = pd.read_csv(url_02, encoding='Latin-1')
df_03.head(5)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,05-07-2003 00:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,07-01-2003 00:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10-10-2003 00:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
